# Lens Maker Equation

Here we create a lens with a fixed spherical shape and compute what its focal length should be, using the formula known as the Lens Maker Equation. Then, we use torchlensmaker forward pass to compute the loss and check that it is close to zero, i.e. that the rays correctly converge to the expected focal point.

Note that it will not be exactly zero because the lens maker equation is an approximation.

In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import matplotlib.pyplot as plt
from itertools import islice

import torchlensmaker as tlm

In [ ]:
# Lens diameter: 40mm
lens_diameter = 40.0

# Circular surface radius
# TODO document sign convention here
R1, R2 = 200, -250

# Indices of refraction
N1, N2 = (1.0, 1.50)

# Lens Maker Equation
expected_focal_length = - 1 / (
    ((N2 - N1) / N1) * (1/R1 - 1/R2)
)

print("Expected focal length (lens's maker equation):", expected_focal_length)

# Surface shapes of the lens
surface1 = tlm.CircularArc(lens_diameter, R1)
surface2 = tlm.CircularArc(lens_diameter, R2)

# Setup the optical stack with incident parallel rays
optics = nn.Sequential(
    tlm.ParallelBeamUniform(lens_diameter, num_rays=20),
    tlm.Gap(10.),

    tlm.RefractiveSurface(surface1, (N1, N2), anchors=("origin", "extent")),
    tlm.Gap(10.),
    tlm.RefractiveSurface(surface2, (N2, N1), anchors=("extent", "origin")),
    
    tlm.Gap(expected_focal_length - 5.),  # TODO is focal length wrt to the first surface?
    tlm.FocalPointLoss(),
)


tlm.render_plt(optics)

# Compute loss with 20 rays
default_input = ((torch.tensor([]), torch.tensor([])), torch.zeros(2))
loss = optics(default_input).item()
print("Loss:", loss)


# Next

* optimize the gap with VariableGap to improve the best fit focal distance estimation
* resample the shape to a parabola to reduce spherical aberation